In [1]:
!nvidia-smi

Mon Jan 31 20:51:15 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd "/content/drive/MyDrive/WPI/Junior/IQP/Machine Models/data/v4"

/content/drive/MyDrive/WPI/Junior/IQP/Machine Models/data/v4


In [4]:
!pip install transformers

     |████████████████████████████████| 3.5 MB 29.3 MB/s 
     |████████████████████████████████| 67 kB 6.8 MB/s 
     |████████████████████████████████| 596 kB 63.0 MB/s 
     |████████████████████████████████| 6.8 MB 59.2 MB/s 
     |████████████████████████████████| 895 kB 63.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [5]:
import json
import torch
import numpy as np
from transformers import BertTokenizerFast, BertForSequenceClassification
from transformers import Trainer, TrainingArguments

In [6]:
# Get device to use
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
model_name = "bert-base-uncased"
# max sequence length for each document/sentence sample
max_length = 512
# load the tokenizer
tokenizer = BertTokenizerFast.from_pretrained(model_name, do_lower_case=True)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [8]:
!ls results/checkpoint-12000

ls: cannot access 'results/checkpoint-12000': No such file or directory


In [9]:
model_path = "results/checkpoint-450"
model = BertForSequenceClassification.from_pretrained(model_path, num_labels=2)
test_trainer = Trainer(model)

In [10]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [11]:
#remove punctuation
import string
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

table = str.maketrans('', '', string.punctuation)

# Remove stop words
from nltk.corpus import stopwords

stop = stopwords.words('english')

# Lemmatize all words
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


In [ ]:
maybe_bad_tweets = {}
maybe_bad_tweets["tweets"] = []
for i in range(1000):
    try:
        followers_timeline_file = open("../followers_timeline_{}.json".format(str(i)), 'r')
        print("Working on file {}".format(str(i)))
    except:
        print("File not found")
        break
    followers_timeline = json.load(followers_timeline_file)
    for user in followers_timeline["followers_timeline"]:
        for follower in followers_timeline["followers_timeline"][user]:
            try:
                tweet_timeline = followers_timeline["followers_timeline"][user][follower]["data"]
                tweet_text_list = []
                for tweet in tweet_timeline:
                        tweet_text = tweet['text']
                        tweet_text = ' '.join([word for word in tweet_text.lower().split() if word not in stop])  # remove stopwords
                        # remove punc.
                        tweet_text = tweet_text.translate(table)
                        tweet_text = ''.join(c for c in tweet_text if not c.isdigit())  # remove numbers
                        tweet_text = tweet_text.replace('  ', ' ')  # remove double spaces
                        tweet_text = ' '.join([lemmatizer.lemmatize(word) for word in tweet_text.split()])  # lemminize
                        tweet_text_list.append(tweet_text)
                X_test_tokenized = tokenizer(tweet_text_list, padding=True, truncation=True, max_length=512)
                test_dataset = Dataset(X_test_tokenized)
                raw_pred, tmp1, tmp2 = test_trainer.predict(test_dataset)
                y_pred = np.argmax(raw_pred, axis=1)
                print(raw_pred)
                print(tmp1)
                print(tmp2)
                counter = 0
                for tweet_pred in y_pred:
                    if tweet_pred == 1:
                        print(tweet_text_list[counter])
                        maybe_bad_tweets["tweets"].append(tweet_timeline[counter])
                    counter += 1
                        
            except:
                pass

Working on file 0


***** Running Prediction *****
  Num examples = 99
  Batch size = 8


***** Running Prediction *****
  Num examples = 91
  Batch size = 8


[[ 4.9882545 -4.579694 ]
 [ 5.013316  -4.6479907]
 [ 4.9869843 -4.5747538]
 [ 4.953843  -4.5793576]
 [ 4.968227  -4.63849  ]
 [ 5.003802  -4.5433784]
 [ 5.0111647 -4.6577816]
 [ 4.994503  -4.6394596]
 [ 4.990772  -4.6661196]
 [ 4.9879456 -4.61366  ]
 [ 4.975474  -4.5362144]
 [ 4.9759164 -4.6548567]
 [ 5.008332  -4.583262 ]
 [ 5.007835  -4.62417  ]
 [ 4.9461427 -4.619702 ]
 [ 5.0006914 -4.6344247]
 [ 5.015901  -4.627749 ]
 [ 5.0144525 -4.6116476]
 [ 4.9869843 -4.5747538]
 [ 4.968795  -4.639729 ]
 [ 4.9810977 -4.596113 ]
 [ 4.7562623 -4.539142 ]
 [ 4.956146  -4.6500006]
 [ 4.9417768 -4.7176104]
 [ 4.9968    -4.619451 ]
 [ 5.0110755 -4.6080422]
 [ 4.904312  -4.586272 ]
 [ 4.979699  -4.6865635]
 [ 5.017619  -4.6322465]
 [ 5.0023613 -4.652549 ]
 [ 5.016928  -4.6509795]
 [ 5.0010653 -4.568719 ]
 [ 4.989798  -4.6543922]
 [ 4.9608316 -4.6178308]
 [ 4.954127  -4.5070696]
 [ 4.9750586 -4.6071463]
 [ 5.005013  -4.461717 ]
 [ 4.932267  -4.609034 ]
 [ 4.8992567 -4.5186872]
 [ 5.003892  -4.649281 ]


***** Running Prediction *****
  Num examples = 100
  Batch size = 8


...

[[ 4.980586  -4.6482153]
 [ 5.0064425 -4.60871  ]
 [ 5.018946  -4.651039 ]
 [ 5.0157447 -4.668914 ]
 [ 4.9882627 -4.6360583]
 [ 5.0048757 -4.675525 ]
 [ 4.9760513 -4.652972 ]
 [ 4.987336  -4.587236 ]
 [ 4.9738445 -4.59774  ]
 [ 4.9990854 -4.635644 ]
 [ 4.9733524 -4.66021  ]
 [ 4.9992166 -4.6048098]
 [ 5.0006013 -4.6334224]
 [ 5.001026  -4.652599 ]
 [ 4.981266  -4.591585 ]
 [ 4.991858  -4.6556444]
 [ 4.9179764 -4.631898 ]
 [ 4.97329   -4.633456 ]
 [ 5.0058627 -4.5861497]
 [ 4.981825  -4.58595  ]
 [ 4.988627  -4.636921 ]
 [ 4.925931  -4.5391846]
 [ 4.770292  -4.435103 ]
 [ 4.9272428 -4.5426445]
 [ 4.9678173 -4.5113344]
 [ 4.9851    -4.671295 ]
 [ 4.9629383 -4.5649624]
 [ 5.008357  -4.5832477]
 [ 4.980007  -4.66406  ]
 [ 4.8022285 -4.4767675]
 [ 4.9271464 -4.7316294]
 [ 4.9990478 -4.633688 ]
 [ 4.95221   -4.6567397]
 [ 4.954089  -4.6151905]
 [ 4.9224863 -4.528272 ]
 [ 4.8295302 -4.4980807]
 [ 4.991152  -4.5242257]
 [ 4.8280416 -4.6039906]
 [ 4.6682167 -4.503785 ]
 [ 4.9760084 -4.624295 ]


***** Running Prediction *****
  Num examples = 15
  Batch size = 8


[[ 4.8864145 -4.7416883]
 [ 4.794773  -4.4489617]
 [ 4.807601  -4.425407 ]
 [ 4.802837  -4.510795 ]
 [ 4.8349786 -4.564276 ]
 [ 4.836627  -4.449048 ]
 [ 4.9076867 -4.552161 ]
 [ 4.7367945 -4.3894653]
 [ 4.7600985 -4.30585  ]
 [ 4.8461156 -4.4798183]
 [ 4.8934817 -4.585307 ]]
None
{'test_runtime': 0.0679, 'test_samples_per_second': 161.949, 'test_steps_per_second': 29.445}


***** Running Prediction *****
  Num examples = 100
  Batch size = 8


[[ 4.7693853 -4.6157365]
 [ 4.7710686 -4.6091275]
 [ 4.784821  -4.610278 ]
 [ 4.80643   -4.634967 ]
 [ 4.765009  -4.6035495]
 [ 4.771409  -4.579391 ]
 [ 4.92013   -4.677204 ]
 [ 4.9452424 -4.6430697]
 [ 4.8173923 -4.6423187]
 [ 4.59576   -4.488773 ]
 [ 4.9329348 -4.7122836]
 [ 4.9060044 -4.666974 ]
 [ 4.7176714 -4.5135417]
 [ 4.93204   -4.6357665]
 [ 4.945592  -4.624361 ]]
None
{'test_runtime': 0.0609, 'test_samples_per_second': 246.207, 'test_steps_per_second': 32.828}


In [ ]:
# Write maybe bad tweets to file
# with open("maybe_bad_tweets_v4.json", 'w') as outfile:
# 	json.dump(maybe_bad_tweets, outfile)